In [9]:
import numpy as np
import pandas as pd
import networkx as nx
from scipy.linalg import eigh
import sys

In [10]:
def load_graph_from_csv(filepath):
    # Try reading with pandas to auto-detect format
    try:
        # First, try reading assuming first column is index
        df = pd.read_csv(filepath, index_col=0)
        # Check if index names match column names (indicates proper format)
        if len(df.index) == len(df.columns):
            node_names = df.columns.tolist()
            data = df.values.astype(float)
        else:
            raise ValueError("Mismatch in dimensions")
    except:
        # If that fails, assume no row labels, just matrix with header row
        df = pd.read_csv(filepath)
        node_names = df.columns.tolist()
        data = df.values.astype(float)

    n = len(node_names)

    # Create weighted graph
    G = nx.Graph()
    for i in range(n):
        for j in range(i+1, n):  # Upper triangle only (undirected)
            if data[i, j] > 0:
                G.add_edge(i, j, weight=data[i, j])

    return G, data, node_names

def compute_all_metrics(G, data):
    """Compute all metrics using NetworkX"""
    n = len(G.nodes())
    results = {}

    print("Computing metrics with NetworkX...")
    print("="*80)

    # 1. Node Degree (unweighted)
    print("1. Node Degree (unweighted)...")
    degrees = dict(G.degree())
    results['Node Degree'] = [degrees[i] for i in range(n)]

    # 2. Clustering Coefficient
    print("2. Clustering Coefficient...")
    clustering = nx.clustering(G)
    results['Clustering Coefficient'] = [clustering[i] for i in range(n)]

    # 3. Betweenness Centrality (weighted)
    print("3. Betweenness Centrality (weighted)...")
    betweenness = nx.betweenness_centrality(G, weight='weight', normalized=True)
    results['Betweenness Centrality'] = [betweenness[i] for i in range(n)]

    # 4. Eigenvector Centrality (weighted)
    print("4. Eigenvector Centrality (weighted)...")
    try:
        eigenvector = nx.eigenvector_centrality(G, weight='weight', max_iter=1000)
        results['Eigenvector Centrality'] = [eigenvector[i] for i in range(n)]
    except:
        print("   WARNING: Eigenvector centrality failed to converge")
        results['Eigenvector Centrality'] = [0.0] * n

    # 5. Local Efficiency (weighted)
    print("5. Local Efficiency (weighted)...")
    local_eff = []
    for node in range(n):
        neighbors = list(G.neighbors(node))
        if len(neighbors) < 2:
            local_eff.append(0.0)
            continue

        # Create subgraph of neighbors
        subgraph = G.subgraph(neighbors)

        # Compute efficiency among neighbors
        total = 0.0
        count = 0
        for i, u in enumerate(neighbors):
            for j, v in enumerate(neighbors):
                if i < j:
                    try:
                        dist = nx.shortest_path_length(subgraph, u, v, weight='weight')
                        if dist > 0:
                            total += 1.0 / dist
                    except nx.NetworkXNoPath:
                        pass
                    count += 1

        local_eff.append(total / count if count > 0 else 0.0)

    results['Efficiency'] = local_eff

    # 6. Removal Tolerance (largest component size change)
    print("6. Removal Tolerance...")
    baseline_size = len(max(nx.connected_components(G), key=len))
    removal_tolerance = []
    for node in range(n):
        G_copy = G.copy()
        G_copy.remove_node(node)
        if len(G_copy.nodes()) > 0:
            largest = len(max(nx.connected_components(G_copy), key=len))
        else:
            largest = 0
        removal_tolerance.append(largest - baseline_size)
    results['Removal Tolerance'] = removal_tolerance

    # 7. Laplacian Spectrum (eigenvalues)
    print("7. Laplacian Spectrum...")
    # Build weighted Laplacian
    degrees_weighted = [sum(data[i, j] for j in range(n) if data[i, j] > 0) for i in range(n)]
    D = np.diag(degrees_weighted)
    L = D - data
    eigenvalues = np.linalg.eigvalsh(L)
    eigenvalues = np.abs(eigenvalues)
    eigenvalues.sort()
    results['Laplacian Spectrum'] = eigenvalues.tolist()

    # 8. Graph-level metrics
    print("8. Graph-level metrics...")

    # Degree Entropy
    total_degree = sum(results['Node Degree'])
    entropy = 0.0
    for deg in results['Node Degree']:
        p = deg / total_degree
        if p > 0:
            entropy -= p * np.log2(p)
    results['Entropy'] = entropy

    # Degree Assortativity
    assortativity = nx.degree_assortativity_coefficient(G)
    results['Assortativity'] = assortativity

    print("="*80)
    print("All metrics computed!\n")

    return results

In [13]:
# File paths
adjacency_file = '/content/newAdjacencyMatrix.csv'
stats_file = '/content/newStats.csv'

print("Network Metrics Verification Tool")
print("Using NetworkX as reference implementation")
print("="*80)
print()

# Load graph
print(f"Loading graph from: {adjacency_file}")
G, data, node_names = load_graph_from_csv(adjacency_file)
print(f"Graph loaded: {G.number_of_nodes()} nodes, {G.number_of_edges()} edges")
print()

# Load C# results
print(f"Loading C# results from: {stats_file}")
csharp_results = pd.read_csv(stats_file, index_col=0)
node_names = csharp_results.columns.tolist()
print(f"Loaded {len(csharp_results)} metrics for {len(node_names)} nodes")
print()

# Compute NetworkX metrics
print(f"Metrics conmuped by this script:\n")
print(compute_all_metrics(G, data))

Network Metrics Verification Tool
Using NetworkX as reference implementation

Loading graph from: /content/newAdjacencyMatrix.csv
Graph loaded: 12 nodes, 22 edges

Loading C# results from: /content/newStats.csv
Loaded 7 metrics for 12 nodes

Metrics conmuped by this script:

Computing metrics with NetworkX...
1. Node Degree (unweighted)...
2. Clustering Coefficient...
3. Betweenness Centrality (weighted)...
4. Eigenvector Centrality (weighted)...
5. Local Efficiency (weighted)...
6. Removal Tolerance...
7. Laplacian Spectrum...
8. Graph-level metrics...
All metrics computed!

{'Node Degree': [2, 4, 5, 4, 5, 3, 2, 5, 5, 3, 4, 2], 'Clustering Coefficient': [1.0, 0.6666666666666666, 0.6, 0.8333333333333334, 0.6, 0.6666666666666666, 1.0, 0.3, 0.5, 0.6666666666666666, 0.5, 1.0], 'Betweenness Centrality': [0.0, 0.0, 0.2727272727272727, 0.0, 0.03636363636363636, 0.0, 0.0, 0.5272727272727272, 0.4909090909090909, 0.0, 0.16363636363636364, 0.0], 'Eigenvector Centrality': [np.float64(0.1665048055